In [1]:
import os

import numpy as np
import pandas as pd

from tensorflow import keras
import tensorflow as tf
import keras.backend as K

from keras.preprocessing.image import ImageDataGenerator


from skimage.morphology import label, binary_opening, disk
from skimage.io import imread
from skimage.transform import resize

In [2]:
#path to data
TEST_FOLDER = 'data/test_v2/'
MODEL_WEIGHTS_PATH = 'models_weigths/u_net_weights_best.h5'
RESULTS_PATH = 'submissions/submission.csv'

IMAGE_RESIZE = (256, 256)
BATCH_SIZE = 32

In [3]:
#defs for decode and encode our masks of ships (Yeap, i took it from kaggle)

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list):
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
    return np.expand_dims(all_masks, -1)

def multi_rle_encode(img):
    labels = label(img[:, :, 0])
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]

# Data generator and Augmentation

In [4]:
#Data generator for images

class generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, data_frame=None, batch_size=BATCH_SIZE, image_size=IMAGE_RESIZE, shuffle=True, predict=False, augment=False):
        self.folder = folder
        self.filenames = filenames
        self.data_frame = data_frame
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.predict = predict
        self.augment = augment
        self.on_epoch_end()
        
    def __load__(self, filename):
        # load jpg file as numpy array
        img = imread(os.path.join(self.folder, filename))
        # create empty mask
        msk = masks_as_image(self.data_frame.query('ImageId==@filename')['EncodedPixels'])
        
        img = resize(img.astype(np.float32), self.image_size, mode='reflect')/255.0     
        msk = resize(msk.astype(np.float32), self.image_size, mode='reflect')
        
        #Image Augmentation
        if self.augment:
            d = np.random.choice([{'zx': 1,
                                   'zy': 1,
                                   'flip_horizontal': False,
                                   'flip_vertical': True},
                                  {'zx': 0.9,
                                   'zy': 0.9,
                                   'flip_horizontal': False,
                                   'flip_vertical': False},
                                  None,
                                  {'zx': 1,
                                   'zy': 1,
                                   'flip_horizontal': True,
                                   'flip_vertical': True},
                                  {'theta': 0,
                                   'zx': 1.1,
                                   'zy': 1.1,
                                   'flip_horizontal': True,
                                   'flip_vertical': False},
                                 ])
            if d:
                img = ImageDataGenerator().apply_transform(img, d)
                msk = ImageDataGenerator().apply_transform(msk, d)
            
        
        return img, msk
    
    def __loadpredict__(self, filename):
        # load jpg file as numpy array
        img = imread(os.path.join(self.folder, filename))
        # resize both image and mask
        img = resize(img.astype(np.float32), self.image_size, mode='reflect')/255.0 
        return img
        
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # predict mode: return images and filenames
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, filenames
        # train mode: return images and masks
        else:
            # load files
            items = [self.__load__(filename) for filename in filenames]
            # unzip images and masks
            imgs, msks = zip(*items)
            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
        
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            # return full batches only
            return int(len(self.filenames) / self.batch_size)

In [5]:
test_filenames = os.listdir(TEST_FOLDER)

#Test generator
test_gen = generator(TEST_FOLDER, test_filenames, predict=True, augment=False, shuffle=False)

# Model U-Net architecture

In [6]:
def double_conv_block(x, n_filters):
    # Conv2D then ReLU activation
    x = tf.keras.layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    # Conv2D then ReLU activation
    x = tf.keras.layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    return x

def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = tf.keras.layers.MaxPool2D(2)(f)
    p = tf.keras.layers.Dropout(0.3)(p)
    return f, p

def upsample_block(x, conv_features, n_filters):
    # upsample
    x = tf.keras.layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    # concatenate
    x = tf.keras.layers.concatenate([x, conv_features])
    # dropout
    x = tf.keras.layers.Dropout(0.3)(x)
    # Conv2D twice with ReLU activation
    x = double_conv_block(x, n_filters)
    return x

In [7]:
inputs = tf.keras.layers.Input((IMAGE_RESIZE[0], IMAGE_RESIZE[1], 3))

# encoder: contracting path - downsample
# 1 - downsample
f1, p1 = downsample_block(inputs, 32)
# 2 - downsample
f2, p2 = downsample_block(p1, 64)
# 3 - downsample
f3, p3 = downsample_block(p2, 128)
# 4 - downsample
f4, p4 = downsample_block(p3, 256)
# 5 - bottleneck
bottleneck = double_conv_block(p4, 512)
# decoder: expanding path - upsample
# 6 - upsample
u6 = upsample_block(bottleneck, f4, 256)
# 7 - upsample
u7 = upsample_block(u6, f3, 128)
# 8 - upsample
u8 = upsample_block(u7, f2, 64)
# 9 - upsample
u9 = upsample_block(u8, f1, 32)
# outputs
outputs = tf.keras.layers.Conv2D(1, 1, padding="same", activation = "sigmoid")(u9)

unet_model = tf.keras.Model(inputs=[inputs], outputs=[outputs], name="U-Net")

2024-02-21 15:26:55.170864: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-21 15:26:55.170886: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-21 15:26:55.170895: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-21 15:26:55.170927: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-21 15:26:55.170943: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
#Load weught to model
unet_model.load_weights(MODEL_WEIGHTS_PATH)

In [9]:
out_pred_rows = []

for imgs, filenames in test_gen:
    
    # predict batch of images
    preds = unet_model.predict(imgs)
    # loop through batch
    for pred, filename in zip(preds, filenames):
        # resize predicted mask
        pred = resize(pred, (768, 768), mode='reflect')
        
        pred = binary_opening(pred>0.5, np.expand_dims(disk(2), -1))
        cur_rles = multi_rle_encode(pred)
        
        if len(cur_rles)>0:
            for c_rle in cur_rles:
                out_pred_rows += [{'ImageId': filename, 'EncodedPixels': c_rle}]
        else:
            out_pred_rows += [{'ImageId': filename, 'EncodedPixels': None}]

1/1 [==============================] - 0s 254ms/step


2024-02-21 15:26:56.707367: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 1s 804ms/step


In [10]:
submission_df = pd.DataFrame(out_pred_rows)[['ImageId', 'EncodedPixels']]
submission_df.to_csv('submission.csv', index=False)